# <center>Generating Music Using LSTM Cells</center>

This workbook will implement modified code from [this](https://github.com/corynguyen19/midi-lstm-gan) GitHub repo.

The idea is to read in MIDI files and convert them to arrays of notes. Then an RNN will be trained to predict the next note. Finally, music is generated by feeding a random string of notes to the RNN and having it iteratively predict the next note to form a song one note at a time.

### Things to test tomorrow:

* Add batch normalization - Loss won't go below ~3
* Add an extra layer (RNN and dense) - Meh
* Normalize between -1 to 1 - Didn't do much
* Decrease step size - MUCH better results with step size of 1
* Mess with batch size - Smaller seems better
* Mash-up 2 songs? - Works surprisingly well
* Converges? - No, different inputs should produce different tracks using the same model
* Try transfer learning and training the last few layers on this data?


### Visualizations
Cross correlations

In [1]:
# Imports
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from music21 import converter, instrument, note, chord, stream, duration
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History

Using TensorFlow backend.


## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [2]:
def get_notes(path):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append([str(element.pitch), element.offset, element.duration])
            elif isinstance(element, chord.Chord):
                song_note = '.'.join(str(n) for n in element.normalOrder)
                song.append([song_note, element.offset, element.duration])
        notes.append(song)

    return notes

def get_notes_with_key(path, filter_key, mode):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
#         Only use music of the same key
        key = midi.analyze('key')
        if(mode==0):
            key_string = str(key.tonic.name)
        elif(mode==1):
            key_string = str(key.mode)
        else:
            key_string = str(key.tonic.name + key.mode)
            
        if(key_string==filter_key):
            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    song.append([str(element.pitch), element.offset, element.duration])
                elif isinstance(element, chord.Chord):
                    song_note = '.'.join(str(n) for n in element.normalOrder)
                    song.append([song_note, element.offset, element.duration])
            notes.append(song)

    return notes

# def get_notes_with_key(path, filter_key, mode):
#     """
#         Gets all notes and chords from midi files where the key matches the string input

#         Parameters
#         ----------
#         path : str
#             The path to the file
#         filter_key : str
#             The string to filter the key on
#         mode : int
#             The type of key used where:
#                 0 - key
#                 1 - major/minor
#                 else - key and major/minor
#     """
#     notes = []

#     for file in glob.glob(path + "*.mid"):        
#         song = []
#         midi = converter.parse(file)
        
#         # Only use music of the same key
#         key = midi.analyze('key')
#         if(mode==0):
#             key_string = str(key.tonic.name)
#         elif(mode==1):
#             key_string = str(key.mode)
#         else:
#             key_string = str(key.tonic.name + key.mode)
            
#         if(key_string==filter_key):
#             print("Parsing %s" % file)
#             notes_to_parse = None

#             try: # file has instrument parts
#                 s2 = instrument.partitionByInstrument(midi)
#                 notes_to_parse = s2.parts[0].recurse() 
#             except: # file has notes in a flat structure
#                 notes_to_parse = midi.flat.notes

#             for element in notes_to_parse:
#                 if isinstance(element, note.Note):
#                     song.append(str(element.pitch))
#                 elif isinstance(element, chord.Chord):
#                     song.append('.'.join(str(n) for n in element.normalOrder))

#     return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 3
input_dir_names = ["test", "Pokemon", "LoZ OOT", "Pokemon GSC", "Pokemon Route", "Undertale", "ABBA", "Beethoven"]

input_path = "../MIDIs/" + input_dir_names[input_dir_choice] + " MIDIs/"
# example of each mode: 0 - C, 1 - major, 2 - Cmajor
# notes = get_notes_with_key(input_path, "Cmajor", 2)
notes = get_notes(input_path)

Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - Cinnabar Island (HGSS Version).mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - S.S. Aqua .mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bicycle.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bug Catching Contest.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Burned Tower.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Champion Battle.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Cherrygrove CityMahogany Town.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dance Theatre.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dark Cave.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dragons Den.mid
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal -

I will now use an algo to determine the key of each song

In [3]:
def print_key(path):
    key_count = dict()
    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        key = midi.analyze('key')
        key_string = key.tonic.name + key.mode
        if (key_string in key_count): 
            key_count[key_string] += 1
        else: 
            key_count[key_string] = 1
        print(key.tonic.name, key.mode)
    return key_count

key_count = print_key(input_path)
# key_count = print_key("../Undertale MIDIs/")
key_count

Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - Cinnabar Island (HGSS Version).mid
G major
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - S.S. Aqua .mid
G major
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
C# major
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bicycle.mid
E minor
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bug Catching Contest.mid
E minor
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Burned Tower.mid
E minor
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Champion Battle.mid
G# minor
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Cherrygrove CityMahogany Town.mid
F major
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dance Theatre.mid
A minor
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dark Cave.mid
A- major
Parsing ../MIDIs/Pokemon GSC MIDIs\Pokemon GoldSilverCryst

{'Gmajor': 5,
 'C#major': 1,
 'Eminor': 3,
 'G#minor': 3,
 'Fmajor': 3,
 'Aminor': 3,
 'A-major': 4,
 'C#minor': 3,
 'Cmajor': 8,
 'Emajor': 4,
 'Dmajor': 8,
 'B-minor': 4,
 'E-major': 3,
 'Bmajor': 2,
 'Amajor': 2,
 'Fminor': 1,
 'Cminor': 1,
 'Dminor': 1,
 'F#major': 1,
 'E-minor': 1}

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [3]:
possibleNotes = set([item[0] for sublist in notes for item in sublist])

# Processing for offsets
possibleOffsets = []
possibleDurations = []

# For each song
for index, song in enumerate(notes):
    song_length = len(song)
    
    # For each note, calculate the difference in offset between this and the previous note
    song_offsets = []
    song_durations = []
    for idx in range(song_length):
        offset = offset = round(song[idx][1] - song[idx - 1][1], 3) if idx > 1 else 0.0
        song_offsets.append(offset)
        if offset not in possibleOffsets:
            possibleOffsets.append(offset)
        
        duration = song[idx][2].quarterLength
        song_durations.append(duration)
        if duration not in possibleDurations:
            possibleDurations.append(duration)
            
    # Update the notes to reflect this
    for idx in range(song_length):
        notes[index][idx][1] = song_offsets[idx]
        notes[index][idx][2] = song_durations[idx]

n_notes = len(possibleNotes)
n_offset = len(possibleOffsets)
n_duration = len(possibleDurations)


possibleNotes = np.array(list(possibleNotes))
possibleOffsets = np.array(list(possibleOffsets))
possibleDurations = np.array(list(possibleDurations))
notes = np.array([list([list(subsublist) for subsublist in sublist]) for sublist in notes])
len(possibleNotes), len(possibleOffsets), len(possibleDurations)

(306, 25, 29)

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [4]:
def prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations):
    """ Prepare the sequences used by the Neural Network """
    song_end_indices = []
    sequence_length = 100
    step_size = 1

    # create a dictionary to map pitches to integers
    pitchnames = sorted(possibleNotes)
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    # create a dictionary to map offset to integers
    offsetnames = sorted(possibleOffsets)
    offset_to_int = dict((offset, number) for number, offset in enumerate(offsetnames))
    
    # create a dictionary to map duration to integers
    durationnames = sorted(possibleDurations)
    duration_to_int = dict((duration, number) for number, duration in enumerate(durationnames))
    
    # find number of each possible choice for normalization
    n_notes = len(possibleNotes)
    n_offset = len(possibleOffsets)
    n_duration = len(possibleDurations)

    network_input = []
    network_output_notes = []
    network_output_offset = []
    network_output_duration = []


    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([np.array([note_to_int[row[0]] / float(n_notes), offset_to_int[row[1]] / float(n_offset), duration_to_int[row[2]] / float(n_duration)]) for row in sequence_in])
            network_output_notes.append(np.array([note_to_int[sequence_out[0]]]))
            network_output_offset.append(np.array([offset_to_int[sequence_out[1]]]))
            network_output_duration.append(np.array([duration_to_int[sequence_out[2]]]))
        song_end_indices.append(len(network_input)-1)


    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 3))

    # Make one-hot-encoding
    network_output_notes = np_utils.to_categorical(network_output_notes, num_classes=n_notes)
    network_output_offset = np_utils.to_categorical(network_output_offset, num_classes=n_offset)
    network_output_duration = np_utils.to_categorical(network_output_duration, num_classes=n_duration)


    return (network_input, network_output_notes, network_output_offset, network_output_duration, song_end_indices)

network_input, network_output_notes, network_output_offset, network_output_duration, song_end_indices = prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations)
network_input.shape

(19486, 100, 3)

## Constructing the model

I will now construct the model using CuDNNLSTM cells because they are significantly faster than regular LSTM cells due to being optimized for CuDA. I will have two CuDNNLSTM layers, followed by two dense layers and a final softmax activation layer to output the most probable result.

Hyperparameters:
* Optimizer - ADAM because it is considered one of the best
* Loss - categorical_crossentropy because it penalizes wrong predictions of multi-class problems best
* Epochs - More epochs are generally better as long as they don't overfit. I track loss over time and have checkpoints every 5 epochs so this will not be a problem
* Batch Size - This determines how many instances should be considered in each batch. Realistically, each different song would interfere with the other so I will reduce this.
    * Smaller Batch Size seems to add more variation

In [5]:
from keras import Input
from keras.models import Model

def create_network(network_input, n_notes, n_offset, n_duration):
    """ create the structure of the neural network """
    input = Input(shape=(network_input.shape[1], network_input.shape[2]))
    lstm_1 = CuDNNLSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True)(input)
    dropout_1 = Dropout(0.3)(lstm_1)
    lstm_2 = Bidirectional(CuDNNLSTM(512, return_sequences=True))(dropout_1)
    dropout_2 = Dropout(0.3)(lstm_2)
    lstm_3 = Bidirectional(CuDNNLSTM(512))(dropout_2)
    dropout_3 = Dropout(0.3)(lstm_3)
    dense_1 = Dense(128)(dropout_3)
    dropout_4 = Dropout(0.3)(dense_1)
    dense_2 = Dense(128)(dropout_4)
    dropout_5 = Dropout(0.3)(dense_2)
    output_notes = Dense(n_notes, activation='softmax')(dropout_5)
    output_offset = Dense(n_offset, activation='softmax')(dropout_5)
    output_duration = Dense(n_duration, activation='softmax')(dropout_5)
    
    model = Model(inputs=input, outputs=[output_notes, output_offset, output_duration])
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], optimizer='adam', loss_weights=[1., 1., 1.])
    return model

# Set up the model
model = create_network(network_input, n_notes, n_offset, n_duration)
history = History()

# Save on each epoch (because training isn't cheap!!!) and can use this to generate music for each checkpoint
outputDest = '../output/LSTM_' + input_dir_names[input_dir_choice] + '_' + str(int(time.time())) + '/'
if not os.path.exists(outputDest):
    os.makedirs(outputDest)

cp_callback = ModelCheckpoint(filepath=outputDest + "LSTMmodel_weights_{epoch:02d}.hdf5",
                              save_weights_only=True,
                              verbose=1,
                              period=10)

# Set parameters
n_epochs = 200
batch_size = 128
model.summary()

W1021 12:45:02.644394 18084 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1021 12:45:02.659955 18084 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1021 12:45:04.298995 18084 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1021 12:45:04.522294 18084 deprecation_wrapper.py:119] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 3)       0                                            
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        (None, 100, 512)     1058816     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 100, 512)     0           cu_dnnlstm_1[0][0]               
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 1024)    4202496     dropout_1[0][0]                  
____________________________________________________________________________________________

## Training the Model

I will save the final model, but keep checkpoints along the way to avoid overfitting and also use these to generate different midis.

In [7]:
model.fit(network_input, [network_output_notes, network_output_offset, network_output_duration], callbacks=[history, cp_callback], epochs=n_epochs, batch_size=batch_size)
model.save(outputDest + 'LSTMmodel_final.h5')

W1021 09:44:37.221143  6432 deprecation.py:323] From C:\Users\Michael\Anaconda3\envs\CITS4404\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
19486/19486 [==============================] - 55s 3ms/step - loss: 8.1915 - dense_3_loss: 4.9146 - dense_4_loss: 1.5388 - dense_5_loss: 1.7381
Epoch 2/200
19486/19486 [==============================] - 48s 2ms/step - loss: 7.5403 - dense_3_loss: 4.6918 - dense_4_loss: 1.2950 - dense_5_loss: 1.5536
Epoch 3/200
19486/19486 [==============================] - 47s 2ms/step - loss: 7.2502 - dense_3_loss: 4.6169 - dense_4_loss: 1.1270 - dense_5_loss: 1.5063
Epoch 4/200
19486/19486 [==============================] - 48s 2ms/step - loss: 7.0680 - dense_3_loss: 4.5486 - dense_4_loss: 1.0542 - dense_5_loss: 1.4653
Epoch 5/200
19486/19486 [==============================] - 48s 2ms/step - loss: 7.0351 - dense_3_loss: 4.5290 - dense_4_loss: 1.0354 - dense_5_loss: 1.4708
Epoch 6/200
19486/19486 [==============================] - 48s 2ms/step - loss: 6.8175 - dense_3_loss: 4.4522 - dense_4_loss: 0.9522 - dense_5_loss: 1.4132
Epoch 7/200
19486/19486 [==============================] - 48s 2

KeyboardInterrupt: 

In [8]:
# Plot the model losses
pd.DataFrame(history.history).plot()
plt.savefig(outputDest + 'LSTM_Loss_per_Epoch.png', transparent=True)
plt.close()

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [6]:
model.load_weights(outputDest + "LSTMmodel_weights_30.hdf5")

In [30]:
def generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations, song_end_indices):
    """ Generate notes from the neural network based on a sequence of notes """
    # create a dictionary to map pitches to integers
    pitchnames = sorted(possibleNotes)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # create a dictionary to map offset to integers
    offsetnames = sorted(possibleOffsets)
    int_to_offset = dict((number, offset) for number, offset in enumerate(offsetnames))
    
    # create a dictionary to map duration to integers
    durationnames = sorted(possibleDurations)
    int_to_duration = dict((number, duration) for number, duration in enumerate(durationnames))
    
    # find number of each possible choice for normalization
    n_notes = len(possibleNotes)
    n_offset = len(possibleOffsets)
    n_duration = len(possibleDurations)
    
    # choose a random point to start
    start = np.random.randint(0, len(song_end_indices)-1)
    pattern = network_input[song_end_indices[start]]
#     start = np.random.randint(0, len(network_input)-1)
#     pattern = network_input[start]
#     np.random.shuffle(pattern)
    sequence_length = pattern.shape[0]
    n_dim = pattern.shape[1]
    
    prediction_output = []
    
    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, sequence_length, n_dim))
        prediction_input = prediction_input

        prediction = model.predict(prediction_input, verbose=0)
        
        note_int = np.argmax(prediction[0])
        note_normalized = note_int / float(n_notes)
        note = int_to_note[note_int]
        
        offset_int = np.argmax(prediction[1])
        offset_normalized = offset_int / float(n_offset)
        offset = int_to_offset[offset_int]
                
        duration_int = np.argmax(prediction[2])
        duration_normalized = duration_int / float(n_duration)
        duration = int_to_duration[duration_int]
        
        result = np.array([note_normalized, offset_normalized, duration_normalized])
        full_prediction = np.array([note, offset, duration])
        
        prediction_output.append(full_prediction)
        pattern = np.append(pattern, result)
        pattern = pattern[3:len(pattern)]
        
    print([str(x[0]) for x in prediction_output])
    
    return prediction_output

prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations, song_end_indices)

['E3', 'B3', 'E3', 'E3', 'E3', 'E4', 'E3', 'E4', 'F#4', 'D4', 'E4', 'B3', '7.11', 'F5', '9.2', '9.2', 'D5', 'D5', 'A3', 'E5', 'G4', 'G4', 'F#4', 'G4', 'A3', 'A3', '6.9', 'A3', '2', 'D5', 'A3', '6.11', 'A4', '6.9', 'A4', 'D4', 'E3', 'D4', 'A3', '9.2', 'A3', 'D5', 'F#4', 'A3', '6.9', '6.9', '2', 'A4', '2', 'D5', 'A3', 'A3', 'A2', 'A3', '2.4.7', 'E3', 'G4', 'A3', '6.11', 'B3', '2', 'E5', 'B2', '2', 'E5', 'D5', '4', '9', 'D5', '2.6', '2.6', 'D4', 'E5', 'D5', 'A3', 'D5', '9.2', 'D5', '6.11', 'E5', '0.4', '2.7', 'A3', 'D5', 'D5', '9', '9', 'D5', 'A3', 'C5', '9.2', 'D5', '2.7', '9.2', 'B2', '6.11', '7.9', '7.9', 'F#5', 'A3', '9.2', 'B3', '6.9', '9.2', 'B2', 'D4', '9.2', 'A3', 'B3', 'G4', 'A3', '2.5', '4.7', 'A3', '2', 'D5', 'D3', 'A3', 'A3', 'D4', 'D4', '9.2', 'A1', 'B3', 'A4', 'D4', 'A3', 'F#5', '9.2', '7.11', '9', '4', '9', '6.9', '6.9', '6.9', 'B3', 'D5', '5.9', '9.1', 'A4', '6.9', '6.9', 'D4', '9.2', 'A3', 'A3', 'E3', 'A4', '6.9', '6.9', 'A3', '9.2', '9.2', 'A4', 'D4', '2.6', 'D4', '2.7',

Next, I will create a midi using these notes and save to a file

In [31]:
from music21 import duration as D

def create_midi(prediction_output, filename, instrument_choice):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []
    output_notes.append(instrument_choice)

    # create note and chord objects based on the values generated by the model
    count = 0
    for pattern in prediction_output:
        note_str = pattern[0]
        offset_str = pattern[1]
        duration_str = pattern[2]
        if "#-" in note_str:# To fix a rare exception using 2 accidentals
            continue
        # pattern is a chord
        if ('.' in note_str) or note_str.isdigit():
            notes_in_chord = note_str.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_note.duration = D.Duration(float(duration_str))
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(note_str)
            new_note.offset = offset
            new_note.duration = D.Duration(float(duration_str))
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += (float(prediction_output[count + 1][1])) if (count + 1 < len(prediction_output)) else 0
        count += 1

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
# Select instrument
instruments = {
    'piano': instrument.Piano(),
    'flute': instrument.Flute(),
    'clarinet': instrument.Clarinet(),
    'ocarina': instrument.Ocarina(),
    'harmonica': instrument.Harmonica(),
    'steel_drum': instrument.SteelDrum(),
    'vocals': instrument.Vocalist(),
    'soprano': instrument.Soprano(),
    'guitar': instrument.Guitar(),
    'elec_guitar': instrument.ElectricGuitar(),
    'violin': instrument.Violin(),
    'saxophone': instrument.Saxophone(),
    'trombone': instrument.Trombone(),
    'trumpet': instrument.Trumpet(),
    'english_horn': instrument.EnglishHorn(),
}
instrument_string = 'piano'
instrument_choice = instruments[instrument_string]

create_midi(prediction_output, outputDest + 'LSTM_output_' + instrument_string + '_8', instrument_choice)
# create_midi(prediction_output, 'lavender')

Or a batch production could be performed of the final model

In [12]:
album_length = 10
for count_output in range(album_length):
    prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations, song_end_indices)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + instrument_string + '_'+ str(count_output), instrument_choice)
    print(f"Created at {outputDest + 'LSTM_output_final' + str(count_output)}")

['0.4.7', 'C3', 'G3', 'E5', 'E3', 'D5', '9.1', '1.4', '2.6', 'D3', '9.2', '6.9', 'D3', 'A3', 'D3', 'D3', '9.2', '2.6', '4.9', 'C3', '7.0', 'C3', 'B-3', '10.2', 'C3', 'C3', '4.7', 'C3', 'C3', '11.2', 'B2', '9.2', '2.6', '6.9', 'B3', 'B3', 'B2', '4.7', '11.2', 'E5', '8.11', 'B2', 'B2', '11.4', 'B3', '7.11', '6.11', '11', 'B-2', '9.11', 'E5', '0.4', 'A2', '9.11', '11.2', '2.6', 'B2', '11.2', '11.2', '11.2', 'B2', 'F6', 'B3', 'B2', '7.11', '7.11', 'A4', '5.10', 'F#4', '4.8', 'F#4', 'E5', 'B2', '7.11', 'A3', 'A3', 'A3', '9', '11.4', 'E5', '4.9', 'C#3', 'E5', '4.9', '7.11', 'E3', '11.4', 'F#5', 'F#5', 'E-6', 'A2', '9.2', 'A5', 'A5', '2.6', 'A4', 'G4', 'G4', 'D5', 'D5', 'F5', 'C5', '9.0', 'B-2', 'F5', 'F5', 'C6', '5.10', 'B-2', '5.10', '11.2', '6.9', 'B2', 'C5', 'D5', 'D5', 'D5', 'C#6', 'F#4', 'A4', 'B4', 'E5', 'B5', 'B4', 'E5', 'B4', 'E5', 'E5', 'G5', 'B5', 'C6', 'C6', 'B5', 'G5', 'D5', 'A5', 'A5', 'F5', 'A5', 'F5', 'F5', 'D5', 'D5', 'D5', 'A4', 'D5', 'F5', 'F5', 'D5', 'B4', 'B4', 'E-5', 'D5

Created at ../output/LSTM_Pokemon GSC_1571622272/LSTM_output_final2
['A4', 'B4', 'C5', 'G4', 'F4', 'C3', 'A4', 'A4', 'E4', 'A4', 'C4', 'A4', 'A4', 'B4', 'C5', 'C5', 'C5', 'C3', 'E5', 'A4', 'E5', 'D5', 'C5', 'B4', 'F3', 'G4', 'A4', 'B4', 'C5', 'G4', 'A4', 'F#3', 'G#4', 'A4', 'B4', 'C5', 'C5', 'D5', 'G3', 'C5', 'B4', 'F5', 'D5', 'G2', 'G3', 'C4', 'F5', 'E5', 'A5', 'G2', 'G3', 'G4', 'C4', 'E4', 'D4', 'D4', 'D4', 'G5', 'C4', 'C5', 'C3', 'G#3', 'G3', 'C5', 'C5', 'G2', 'F5', 'C3', 'B-5', 'G2', 'G2', 'C6', 'B-2', 'G5', 'A5', 'B-2', 'A5', 'A5', 'C5', '8.11', 'G#5', 'G5', 'D5', 'D5', 'D5', 'G5', 'D5', 'D5', 'D5', 'C5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'C5', 'D5', 'D5', 'E5', 'D5', 'E5', 'C5', 'C5', 'D5', 'F5', 'G5', 'G5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'G5', 'D5', 'D6', 'E5', 'C5', 'E5', 'C5', 'D5', 'C#6', 'C5', 'D5', 'B4', 'E5', 'G5', 'D5', 'E5', 'G5', 'D6', 'B4', 'G5', 'D5', 'G5', 'G5', 'E5', 'D5', 'G5', 'E5', 'G5', 'C#5', 'D5', 'D5', 'D5', 'G5', 'D5', 'D5', 'D5', 'D5', 'C5', 'D

Created at ../output/LSTM_Pokemon GSC_1571622272/LSTM_output_final5
['D5', 'F#5', 'D5', 'F#5', 'B5', 'C5', 'F#5', 'D5', 'F#5', 'C6', 'D5', 'G5', 'B4', 'F#5', 'C6', 'D5', 'G5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D5', 'D6', 'A5', 'A5', 'G5', 'A5', 'A5', 'A5', 'A5', 'A5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'A5', 'F#5', 'E5', 'F#5', 'F#5', 'A5', 'F#5', 'E5', 'E5', 'F#5', 'D5', 'D5', 'D5', 'D5', 'F#5', 'F#5', 'D5', 'C#5', 'F#5', 'B5', 'C5', 'G5', 'D5', 'F#5', 'F#5', 'G5', 'F#5', 'F#5', 'F#5', 'B-3', 'G5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'F#5', 'D5', 'C5', 'D5', 'D5', 'C5', 'D5', 'F#5', 'C5', 'F#5', 'D5', 'C5', 'D5', 'A3', 'C5', 'G5', 'A3', 'C5', 'D5', 'C#3', 'G5', 'D5', 'A3', 'C5', 'E5', 'A3', 'F#5', 'F#5', 'A3', 'F5', 'F5', 'A3', 'F5', 'F5', 'G#4', 'C5', 'D5', 'G4', 'C5', 'F#5', 'G4', 'C5', 'F#5', 'A4', 'F5', 'F5', 'G#4', 'C5', 'F5', 'G#4', 'C5', 'F5', 'G#4', 'C5', 'F#5', 'G#4', 'C5'

['6.9', '1.4', 'B2', 'B2', 'B2', '10.2', '9.1.4', 'B4', 'G2', '7.11', '9.1', '11.2', '2.6', 'G2', 'G2', '1.4', 'A2', '11.2', 'A2', 'C#6', 'B5', '9.1.4', '7.9.11', '6.9', 'F#2', '1.6', '6.9', '4', 'B2', 'B2', 'B2', 'F#6', 'B-2', 'G2', 'A2', 'E6', 'F#6', '4.8', 'E5', 'B4', 'E5', 'G#5', 'G#5', 'E3', 'B-5', 'E3', 'B5', 'E3', 'B3', '2.8', '4.6', 'F#6', '5.8', 'F5', 'B4', 'F5', 'G#5', 'G#5', 'F3', 'B-5', 'F3', 'B5', 'F3', 'C#4', '10', '8.11', 'C#6', 'C#5', 'F#3', 'B4', 'F#4', 'C#5', 'B4', 'F#4', '1.6', '6.11', 'F#4', 'F#2', 'C#4', 'F#4', 'F#2', 'B-4', 'C#5', 'F#3', 'F#5', '6.10', '8.1', '6.10.1', '8.9.0', '8.9.11', '6.10.1', 'E-5', 'B-2', 'E5', 'F#5', 'B2', 'B3', 'C#4', 'E-4', '3.6', '4.8', '3.6', '4.6.11', '3.6.8', '6.11', 'E5', '1', 'G#4', 'E4', 'C#5', 'C#5', 'G#4', 'E4', 'C#3', 'C5', 'C#3', 'B4', 'B4', 'C#3', '8.10', 'C#5', 'G#2', 'E-5', 'E5', 'F#2', 'C#4', 'E4', 'B-4', '4.10', 'F#2', '1.6', 'F#2', '4.10', 'F#2', '3.6', '1.4.6', '1.4', '3', 'B2', 'E4', 'B2', 'B2', 'F4', '5.11', '6.11', 'E

Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [23]:
# Have each model make a song
count = 0
filepaths = glob.glob(outputDest + "*.hdf5")
for model_path in filepaths:
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1